### Lecture 2

In [1]:
import torch
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
torch.cuda.is_available()

False

In [3]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')


device

device(type='cpu')

In [4]:
noise_dim = 100
label_dim = 10

#### Generator Model

In [5]:
class Generator(torch.nn.Module):
    
    def __init__(self):
        
        super(Generator, self).__init__()
        
        self.fcn = torch.nn.Sequential(
            # Fully Connected Layer 1
            torch.nn.Linear(
                in_features=noise_dim + label_dim,
                out_features=240,
                bias=True
            ),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            # Fully Connected Layer 2
            torch.nn.Linear(
                in_features=240,
                out_features=240,
                bias=True
            ),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            # Fully Connected Layer 3
            torch.nn.Linear(
                in_features=240,
                out_features=240,
                bias=True
            ),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            # Fully Connected Layer 4
            torch.nn.Linear(
                in_features=240,
                out_features=240,
                bias=True
            ),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            # Fully Connected Layer 5
            torch.nn.Linear(
                in_features=240,
                out_features=240,
                bias=True
            ),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            # Fully Connected Layer 6
            torch.nn.Linear(
                in_features=240,
                out_features=784,
                bias=True
            ),
            torch.nn.Sigmoid()
        )

    def forward(self, batch, labels):
        inputs = batch.view(batch.size(0), -1)
        ret = torch.cat((inputs, labels), dim=1)
        ret = self.fcn(ret)
        return ret

#### Maxout Activation

##### Source: https://github.com/pytorch/pytorch/issues/805

In [6]:
class Maxout(torch.nn.Module):

    def __init__(self, num_pieces):

        super(Maxout, self).__init__()

        self.num_pieces = num_pieces

    def forward(self, x):

        # x.shape = (batch_size? x 625)

        assert x.shape[1] % self.num_pieces == 0  # 625 % 5 = 0

        ret = x.view(
            *x.shape[:1],  # batch_size
            x.shape[1] // self.num_pieces,  # piece-wise linear
            self.num_pieces,  # num_pieces
            *x.shape[2:]  # remaining dimensions if any
        )
        
        # ret.shape = (batch_size? x 125 x 5)

        # https://pytorch.org/docs/stable/torch.html#torch.max        
        ret, _ = ret.max(dim=2)

        # ret.shape = (batch_size? x 125)

        return ret

#### Discriminator Model

In [7]:
class Discriminator(torch.nn.Module):
    
    def __init__(self):
        
        super(Discriminator, self).__init__()
        
        self.fcn = torch.nn.Sequential(
            # Fully Connected Layer 1
            torch.nn.Linear(
                in_features=784 + label_dim,
                out_features=240,
                bias=True
            ),
            Maxout(5),
            torch.nn.Dropout(0.5),
            # Fully Connected Layer 2
            torch.nn.Linear(
                in_features=48,
                out_features=240,
                bias=True
            ),
            Maxout(5),
            torch.nn.Dropout(0.5),
            # Fully Connected Layer 3
            torch.nn.Linear(
                in_features=48,
                out_features=1,
                bias=True
            ),
            torch.nn.Sigmoid()
        )

    def forward(self, batch, labels):
        ret = batch.view(batch.size(0), -1)
        ret = torch.cat((ret, labels), dim=1)
        ret = self.fcn(ret)
        return ret

#### MNIST Dataset

In [8]:
import torchvision

In [9]:
class FlattenTransform:
    
    def __call__(self, inputs):
        return inputs.view(inputs.shape[0], -1)


data_train = torchvision.datasets.FashionMNIST(
    './data/fashionMNIST',
    train=True,
    download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        FlattenTransform()
    ])
)

Extracting ./data/fashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/fashionMNIST/FashionMNIST/raw


Extracting ./data/fashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/fashionMNIST/FashionMNIST/raw


Extracting ./data/fashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/fashionMNIST/FashionMNIST/raw


Extracting ./data/fashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/fashionMNIST/FashionMNIST/raw
Processing...


/home/stephany/miniconda3/envs/pytorch_env/lib/python3.6/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729004493/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [10]:
BATCH_SIZE = 64

train_loader = torch.utils.data.DataLoader(
    data_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

#### Optimizer

In [11]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)


discriminator_optimizer = torch.optim.SGD(
    discriminator.parameters(),
    lr=0.003,
    momentum=0.5
#     dampening=0.0001
)

generator_optimizer = torch.optim.SGD(
    generator.parameters(),
    lr=0.001,
    momentum=0.5
#     dampening=0.0001
)

criterion = torch.nn.BCELoss()

#### Optimizer Scheduler

In [12]:
'''
discriminator_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer=discriminator_optimizer,
    step_size=1,
    gamma=0.99,
    last_epoch=-1
)

generator_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer=generator_optimizer,
    step_size=1,
    gamma=0.99,
    last_epoch=-1
)
'''

'\ndiscriminator_scheduler = torch.optim.lr_scheduler.StepLR(\n    optimizer=discriminator_optimizer,\n    step_size=1,\n    gamma=0.99,\n    last_epoch=-1\n)\n\ngenerator_scheduler = torch.optim.lr_scheduler.StepLR(\n    optimizer=generator_optimizer,\n    step_size=1,\n    gamma=0.99,\n    last_epoch=-1\n)\n'

#### Lambda Learning Rate

In [13]:
def decayLR(e):
    if e < 200:
        return 1
    elif e < 400:
        return 1 / (1 + e * 0.1)
    elif e < 600:
        return 1 / (1 + e * 0.001)
    elif e < 800:
        return 1 / (1 + e * 0.00001)
    elif e < 1000:
        return 1 / (1 + e * 0.0000001)
    else:
        return 1 / (1 + e * 0.000000001)


discriminator_scheduler = torch.optim.lr_scheduler.LambdaLR(
    discriminator_optimizer,
    decayLR
)

generator_scheduler = torch.optim.lr_scheduler.LambdaLR(
    generator_optimizer,
    decayLR
)

#### Visualize Function

In [14]:
def visualizeGAN(tgt_pth, images, labels, epoch):

    categories = [
        'T-shirt/top',
        'Trouser',
        'Pullover',
        'Dress',
        'Coat',
        'Sandal',
        'Shirt',
        'Sneaker',
        'Bag',
        'Ankle boot'
    ]

    fig, axes = plt.subplots(2, 5, figsize=(20, 18))
    
    fig.suptitle('Epoch {}'.format(str(epoch).zfill(4)))

    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            cell.imshow(
                images[row * 5 + col],
                cmap='gray'
            )
            
            cell.set_title('{}'.format(
                categories[torch.argmax(labels[row * 5 + col])]
            ))

            cell.axis("off")


    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(epoch).zfill(3))))
    
    plt.close()

#### Onehot Encoding

In [15]:
def encodeOneHot(labels):
    ret = torch.FloatTensor(labels.shape[0], label_dim)
    ret.zero_()
    ret.scatter_(dim=1, index=labels.view(-1, 1), value=1)
    return ret

#### Train GANs

In [16]:
real_labels = torch.ones(BATCH_SIZE, 1).to(device)
fake_labels = torch.zeros(BATCH_SIZE, 1).to(device)

test_z = (2 * torch.randn(10, noise_dim) - 1).to(device)
test_y = encodeOneHot(torch.tensor(np.arange(0, 10))).to(device)

num_epochs = 2048
num_steps = len(train_loader) // BATCH_SIZE

In [17]:
import os

visuals_dir = 'visuals-section-3-lecture-2-fashion-mnist-exercise-b'

if not os.path.exists(visuals_dir):
    os.mkdir(visuals_dir)

In [ ]:
d_loss_ls = []
g_loss_ls = []
d_lr_ls = []
g_lr_ls = []


for epoch in range(num_epochs):
    
    # Loss Log
    d_counter = 0
    g_counter = 0
    d_loss = 0
    g_loss = 0

    for i, (images, labels) in enumerate(train_loader):

        if i == num_steps:
            break

        # Train Discriminator
        for _ in range(4):
        
            real_images = images.to(device)
            real_conditions = encodeOneHot(labels).to(device)

            fake_conditions = encodeOneHot(torch.randint(0, 10, (BATCH_SIZE,))).to(device)

            fake_images = generator(
                (2 * torch.randn(BATCH_SIZE, noise_dim) - 1).to(device),
                fake_conditions
            )

            discriminator_optimizer.zero_grad()

            real_outputs = discriminator(real_images, real_conditions)
            fake_outputs = discriminator(fake_images, fake_conditions)

            d_x = criterion(real_outputs, real_labels)
            d_g_z = criterion(fake_outputs, fake_labels)

            d_x.backward()
            d_g_z.backward()

            discriminator_optimizer.step()
            
            # Loss Log
            d_counter += 1
            d_loss = d_x.item() + d_g_z.item()


        # Train Generator
        z = (2 * torch.randn(BATCH_SIZE, noise_dim) - 1).to(device)
        y = encodeOneHot(torch.randint(0, 10, (BATCH_SIZE,))).to(device)

        generator.zero_grad()

        outputs = discriminator(generator(z, y), y)

        loss = criterion(outputs, real_labels)

        loss.backward()

        generator_optimizer.step()
        
        # LR Decay
        discriminator_scheduler.step()
        generator_scheduler.step()
        
        # Loss Log
        g_counter += 1
        g_loss += loss.item()

    # Loss Log
    if epoch % 10 == 0:
        print(
            'e:{}, G:{:.3f}, D:{:.3f}, G_LR:{}, D_LR:{}'.format(
                epoch,
                g_loss / g_counter,
                d_loss / d_counter,
                generator_scheduler.get_lr(),
                discriminator_scheduler.get_lr()
            )
        )
    
    # Loss Log for Plot
    g_loss_ls.append(g_loss / g_counter)
    d_loss_ls.append(d_loss / d_counter)
    
    # Learning Rate Decay Log
    g_lr_ls.append(generator_scheduler.get_lr())
    d_lr_ls.append(discriminator_scheduler.get_lr())


    # Visualize Results
    if epoch % 5 == 0:

        generated = generator(test_z, test_y).detach().cpu().view(-1, 28, 28)

        visualizeGAN(visuals_dir, generated, test_y, epoch)

/home/stephany/miniconda3/envs/pytorch_env/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


e:0, G:1.014, D:0.020, G_LR:[0.001], D_LR:[0.003]
e:10, G:6.251, D:0.002, G_LR:[0.001], D_LR:[0.003]
e:20, G:7.132, D:0.001, G_LR:[3.289473684210526e-05], D_LR:[9.868421052631579e-05]
e:30, G:7.626, D:0.001, G_LR:[0.0006973500697350071], D_LR:[0.002092050209205021]
e:40, G:8.551, D:0.001, G_LR:[0.0006353240152477764], D_LR:[0.0019059720457433292]
e:50, G:8.166, D:0.000, G_LR:[0.000992910618186151], D_LR:[0.002978731854558453]
e:60, G:2.716, D:0.006, G_LR:[0.0009999146072925374], D_LR:[0.002999743821877612]
e:70, G:2.712, D:0.008, G_LR:[0.0009999006098793779], D_LR:[0.002999701829638134]
e:80, G:3.267, D:0.003, G_LR:[0.000999998866001286], D_LR:[0.0029999965980038582]


In [ ]:
# Visualize Results
generated = generator(test_z, test_y).detach().cpu().view(-1, 28, 28)

visualizeGAN(visuals_dir, generated, test_y, epoch)

#### Visualize Loss

In [ ]:
fig = plt.figure(figsize=(16, 10))
plt.plot(d_loss_ls, label='D Loss')
plt.plot(g_loss_ls, label='G Loss')
plt.legend()
plt.show();

#### Visualize Learning Rate Decay

In [ ]:
fig = plt.figure(figsize=(16, 10))
plt.plot(d_lr_ls, label='D LR')
plt.plot(g_lr_ls, label='G LR')
plt.legend()
plt.show();

#### Google Collaboratory

Notebook: https://colab.research.google.com/drive/17A_Ecl2sZdXra_YSnt7F-F4Z8vCa2U2J